In [1]:
# Import
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import edward as ed
import numpy as np
import tensorflow as tf

from edward.models import Bernoulli, Multinomial, Beta, Dirichlet, PointMass
from observations import karate
from sklearn.metrics.cluster import adjusted_rand_score

import networkx as nx
import matplotlib.pyplot as plt

import inspect

In [2]:
# Set seed for both NumPy and TensorFlow
ed.set_seed(42)

# Karate Club Toy Example Data Set
# 1) X_data : 34 x 34 Adjacency Matrix denoting the connections (friendships)
# between the members of the Karate Club

# 2) Z_true : 1 x 34 Vector denoting the membership
# (0 for Mr. Hi, 1 for Officer)

X_data, Z_true = karate("~/data")

# Output
print("X_data = ")
print(X_data, "\n")

print("Z_true = ")
print(Z_true)

X_data = 
[[0 1 1 ... 1 0 0]
 [1 0 1 ... 0 0 0]
 [1 1 0 ... 0 1 0]
 ...
 [1 0 0 ... 0 1 1]
 [0 0 1 ... 1 0 1]
 [0 0 0 ... 1 1 0]] 

Z_true = 
[0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 1 0 0 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1]


In [3]:
# Number of Vertices N
N = X_data.shape[0]

# Define the a priori value of communities
# (only 2 since Mr. Hi vs. Officer)
K = 2

In [4]:
# Model

# Gamma: Dirichlet Distributed Random Variable
# Argument(s): Concentration = alpha is a 2 x 1 tensor of ones and represents
# the mean total counts of class occurence
gamma = Dirichlet(concentration = tf.ones([K]))

# Pi: Beta Distributed Random Variable
# Argument(s): concentration0 = alpha, concentration1 = beta which represent mean
# total counts of a 0 or 1, respectively.
Pi = Beta(concentration0=tf.ones([K, K]), concentration1=tf.ones([K, K]))

# Z: Dirichlet Multinomial Compound Distribution
Z = Multinomial(total_count=1.0, probs=gamma, sample_shape=N)

# X: Bernoulli Distribution
X = Bernoulli(probs=tf.matmul(Z, tf.matmul(Pi, tf.transpose(Z))))

In [5]:
# Testing
qgamma = PointMass(params=tf.nn.softmax(tf.get_variable("qgamma/params", [K])))
qPi = PointMass(params=tf.nn.sigmoid(tf.get_variable("qPi/params", [K, K])))
qZ = PointMass(params=tf.nn.softmax(tf.get_variable("qZ/params", [N, K])))

inspect.getsource(PointMass)


IOError: could not find class definition

In [ ]:
# Inference & Expectation Maximization
qgamma = PointMass(tf.nn.softmax(tf.get_variable("qgamma/params", [K])))
qPi = PointMass(tf.nn.sigmoid(tf.get_variable("qPi/params", [K, K])))
qZ = PointMass(tf.nn.softmax(tf.get_variable("qZ/params", [N, K])))

# MAP Optimization
inference = ed.MAP({gamma: qgamma, Pi: qPi, Z: qZ}, data={X: X_data})
inference.initialize(n_iter=250)

# Global Variable Initialization
tf.global_variables_initializer().run()

In [20]:
x = PointMass(params=20)

RandomVariable("PointMass_4/", shape=(), dtype=int32)


In [5]:
# Inference & Expectation Maximization
qgamma = PointMass(tf.nn.softmax(tf.get_variable("qgamma/params", [K])))
qPi = PointMass(tf.nn.sigmoid(tf.get_variable("qPi/params", [K, K])))
qZ = PointMass(tf.nn.softmax(tf.get_variable("qZ/params", [N, K])))

# MAP Optimization
inference = ed.MAP({gamma: qgamma, Pi: qPi, Z: qZ}, data={X: X_data})
inference.initialize(n_iter=250)

# Global Variable Initialization
tf.global_variables_initializer().run()

edward/util/random_variables.py:52: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  not np.issubdtype(value.dtype, np.float) and \
edward/util/random_variables.py:53: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  not np.issubdtype(value.dtype, np.int) and \


In [6]:
for _ in range(inference.n_iter):
    info_dict = inference.update()
    inference.print_progress(info_dict)


250/250 [100%] ██████████████████████████████ Elapsed: 2s | Loss: 340.618

In [7]:
# CRITICISM
Z_pred = qZ.mean().eval().argmax(axis=1)
print("Result (label flip can happen):")
print("Predicted")
print(Z_pred)
print("True")
print(Z_true)
print("Adjusted Rand Index =", adjusted_rand_score(Z_pred, Z_true))

Result (label flip can happen):
Predicted
[0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0]
True
[0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 1 0 0 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1]
Adjusted Rand Index = -0.00635324015248


[1 1 0 1 1 0 1 1 1 1]
[1 1 1 1 1 1 1 1 0 1]


-0.129032258065
